# Introduction

In this notebook we demonstrate the use of **BM25 (Best Matching 25)** Information Retrieval technique to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each test case content as an entire document that must be returned to the query made


# Import Libraries

In [1]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

import pandas as pd

from modules.models_runner.feat_br_runner import Feat_BR_Runner
from modules.utils import aux_functions

from IPython.display import display

import warnings; warnings.simplefilter('ignore')

# Running BM25 Model

In [2]:
%%time

runner = Feat_BR_Runner()
bm25_model, bm25_eval = runner.run_bm25_model()

Features.shape: (21, 8)
SelectedBugReports2.shape: (93, 22)
Expert and Volunteers Matrix.shape: (21, 93)

Model Evaluation -------------------------------------------
{'Measures': {'Mean FScore of BM25_Model_AllData': 0.03225806451612901,
              'Mean Precision of BM25_Model_AllData': 0.016897081413210457,
              'Mean Recall of BM25_Model_AllData': 0.3548387096774194},
 'Setup': [{'Name': 'BM25_Model_AllData'},
           {'Top Value': 100},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class 'modules.utils.tokenizers.PorterStemmerBased_Tokenizer'>}]}
CPU times: user 618 ms, sys: 8.35 ms, total: 626 ms
Wall time: 622 ms


In [3]:
aux_functions.highlight_df(runner.orc.iloc[0:20, 0:7])

NameError: name 'aux_functions' is not defined

In [ ]:
aux_functions.highlight_df(bm25_model.get_trace_links_df().iloc[0:20, 0:7])

In [ ]:
aux_functions.highlight_df(bm25_model.get_sim_matrix().iloc[0:20, 0:7])